In [20]:
##Import necessary Modules -- pip install rasterio geopandas pandas rasterstats os logging
import rasterio
import geopandas as gpd
import pandas as pd
import numpy as  np
from rasterstats import zonal_stats
import os
import logging ##-- i am trying a new approach to use logging to track what's happening in your code while it runs


In [21]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

## Source - stackoverflow.com/questions/50714316/how-to-use-logging-getlogger-name-in-multiple-modules

In [26]:
##creating function step by step

# no this is not chatgpt lol--i am doing it myself--First function to allign CRS

def check_and_align_crs(vector_gdf, raster_path):

    with rasterio.open(raster_path) as src:
        raster_crs = src.crs

    # convert CRS to string format for comparison 
    vector_crs = str(vector_gdf.crs)
    raster_crs = str(raster_crs)

    logger.info(f"Vector CRS: {vector_crs}")
    logger.info(f"Raster CRS: {raster_crs}")

    if vector_crs != raster_crs:
        logger.warning("CRS mismatch detected. Reprojecting vector data...")
        vector_gdf = vector_gdf.to_crs(raster_crs) ## --reprojecting if CRS mismatch occurs
        logger.info("Reprojection completed")
    
    return vector_gdf
## acutal function to calucalte zonal_stats, -- I added id column as an input because different shapefiles/geojson files might use different column names as their unique identifier.

def calculate_zonal_stats(raster_path, vector_path, id_column):
 
    try:
        # verify files exist
        if not os.path.exists(raster_path):
            raise FileNotFoundError(f"Raster file not found: {raster_path}")
        if not os.path.exists(vector_path):
            raise FileNotFoundError(f"Vector file not found: {vector_path}")
            
        # read the vector file
        logger.info("Reading vector file...")
        gdf = gpd.read_file(vector_path)
        
        # verify the ID column exists
        if id_column not in gdf.columns:
            raise ValueError(f"ID column '{id_column}' not found in the vector file")
        
        # check and align CRS
        gdf = check_and_align_crs(gdf, raster_path)
            
        # calculate zonal statistics
        logger.info("Calculating zonal statistics...")
        stats = zonal_stats(gdf, 
                          raster_path,
                          stats=['mean', 'std'],
                          geojson_out=True)
        
        # convert to DataFrame
        logger.info("Processing results...")
        result_df = pd.DataFrame([
            {
                'ID': feature['properties'][id_column],
                'mean': feature['properties']['mean'],
                'std': feature['properties']['std']
            }
            for feature in stats
        ])
        
        # set ID as index
        result_df.set_index('ID', inplace=True)
        
        logger.info("Calculation completed successfully!")
        return result_df
        
    except Exception as e:
        logger.error(f"Error occurred: {str(e)}")
        return None

def main():
    # define paths
    base_path = r"C:\Aviskar_Portal\Aviskar_Confidential\Geospattia)_Analytics\lab_6\task_2"
    raster_path = os.path.join(base_path, "ndvi.tif")
    vector_path = os.path.join(base_path, "grid.geojson")
    output_path = os.path.join(base_path, "zonal_stats_results.csv")
    
    # calculate zonal statistics
    logger.info("Starting zonal statistics calculation...")
    results = calculate_zonal_stats(raster_path, vector_path, id_column='id')
    
    if results is not None:
        logger.info("\nZonal Statistics Results:")
        print(results.head(5))##--only taking top 5 results to show here. Check the csv file for full data
        
        # save results to CSV
        results.to_csv(output_path)
        logger.info(f"Results saved to: {output_path}")
        
        # basic statistics summary
        logger.info("\nSummary Statistics:")
        print(results.describe())
        
        logger.info(f"Total number of zones processed: {len(results)}")

if __name__ == "__main__":
    main()

2024-11-18 17:05:44,350 - INFO - Starting zonal statistics calculation...
2024-11-18 17:05:44,352 - INFO - Reading vector file...
2024-11-18 17:05:44,419 - INFO - Vector CRS: EPSG:32615
2024-11-18 17:05:44,420 - INFO - Raster CRS: EPSG:32615
2024-11-18 17:05:44,421 - INFO - Calculating zonal statistics...
2024-11-18 17:05:49,483 - INFO - Processing results...
2024-11-18 17:05:49,484 - INFO - Calculation completed successfully!
2024-11-18 17:05:49,486 - INFO - 
Zonal Statistics Results:
2024-11-18 17:05:49,490 - INFO - Results saved to: C:\Aviskar_Portal\Aviskar_Confidential\Geospattia)_Analytics\lab_6\task_2\zonal_stats_results.csv
2024-11-18 17:05:49,490 - INFO - 
Summary Statistics:
2024-11-18 17:05:49,494 - INFO - Total number of zones processed: 372


        mean       std
ID                    
41  0.405855  0.067818
42  0.406319  0.058588
61  0.424627  0.065736
62  0.417399  0.071759
63  0.363553  0.140118
             mean         std
count  372.000000  372.000000
mean     0.352218    0.104724
std      0.047928    0.026245
min      0.194923    0.042383
25%      0.318571    0.088968
50%      0.354612    0.105973
75%      0.390165    0.117623
max      0.444366    0.185824
